# Synchrotron Radiation

## Pre-Requisites
As usual, we start with importing the necessary modules.

In [ ]:
from astropy import units as u
from astropy import constants as c

from astropy.modeling.powerlaws import PowerLaw1D

import numpy as np

import matplotlib.pyplot as plt

from math import pi
from numpy import sqrt, cos, sin, tan, abs

## Relativistic Gyro-Frequency

Let's declare a function to calculate the gyro-frequency, as on the lecture slides on page 27. It takes as parameters the magnetic field B, the Lorentz factor $\gamma$, the charge q in elementary charges with -1 as default and the mass of the particle with the electron mass as default.

In [ ]:
def omega_r(B, gamma, q = -1, m = c.m_e) :
    
    # convert q into charge
    q *= c.e.si
    
    # we only need the magnitude of the charge:
    q = abs(q)
    
    # equation as on the lecture slides
    omega_r = q*B/(gamma*m)
    
    # return omega_r in units of Hz
    return omega_r.to(u.Hz)

This function takes the Lorentz factor $\gamma$ as parameter. But often we have the total energy of a particle, we can calculate the Lorentz factor with
$$ \gamma = \frac{E_\mathrm{tot}}{m c^2} .$$
Let's write another function for that. It will take the (total) energy of the particle and the particle mass as parameter. The mass defaults to the electron mass.

In [ ]:
def gamma_fromEtot (E, m = c.m_e) :
    
    gamma = E/(m*c.c**2)
    
    # gamma is dimensionless, let's decompose before returning
    return gamma.decompose()

The lecture example is for an electron with kinetic energy of

In [ ]:
E_kin = 5e-10 *u.J

We need the total energy, which is the sum of the kinetic energy and the rest energy ($mc^2$).

In [ ]:
E_tot = E_kin + c.m_e*c.c**2

In [ ]:
E_tot

You see that at these energies the rest energy of the electron is negligible.

Let's calculate the Lorentz factor:

In [ ]:
gamma_fromEtot(E_tot)

The magnetic field is still

In [ ]:
B = 40 * u.microTesla

The gyro-frequency is

In [ ]:
omega = omega_r(B, gamma_fromEtot(E_tot))

In [ ]:
omega

The number of cycles is

In [ ]:
omega / (2*pi)

We can plot $\omega_r$ depending on the Lorentz factor.

In [ ]:
gammas = np.logspace(0,4,41)

omegas = omega_r(B, gammas)

plt.loglog(gammas, omegas)

plt.xlabel('$\gamma$')
plt.ylabel('$\omega _r$ [{}]'.format(omegas.unit))

## Synchrotron Frequency

The synchrotron frequency is (see slide 40)
$$\nu_\mathrm{sync} = \frac{1}{2\pi} \left( \frac{U}{m c^2} \right)^2 \frac{qB}{m} \sin \phi$$
We are interested in electrons only, so we can keep the charge and mass fixed. Here is a function to do the calculation:

In [ ]:
def nu_sync (U, B, phi = 90*u.degree) : 
    
    ret = U**2 * c.e.si * B * sin(phi) / 2 / pi / c.m_e**3 / c.c**4
    
    return ret.decompose().to(u.Hz)

The lecture example takes an assumed magnetic field in the Crab Nebula of $B = 5 \times 10^{-8}\,\mathrm{T}$.
We have seen that there are electrons with an energy of $U = 21\,\mathrm{TeV}$. The synchrotron frequency is

In [ ]:
nu = nu_sync(21*u.TeV, 5e-8*u.T)

In [ ]:
nu

And the photon energy is

In [ ]:
E = nu*c.h

In [ ]:
E.to(u.keV)

### Exercise
In order to fill the table on the tutorial sheet you need a function to calculate the electron energy $U$ in eV from the synchrotron frequency. Write a function to do this calculation.

In [ ]:
def Eelectron_sync (nu, B, phi = 90*u.degree) :
    
    # fill in your code here
    E = 0*u.eV
    
    return E.to(u.eV)

Now we test the function.

In [ ]:
Eelectron_sync(9.8*u.keV/c.h, B)

## Power Spectrum
We want to write a function to calculate the power spectrum of the synchrotron radiation of a mono-energetic electron. The equation (see slide 43) is
$$ P(\omega) = \frac{\sqrt{3}}{8 \pi^2} \frac{q^3 B \sin \phi}{\epsilon_0 c m} F\left( \frac{\omega}{\omega_c} \right)$$
with
$$ F(x) = x \int_x^\infty K_{5/3} (z) dz $$
and $K_{5/3} (z)$ being a modified Bessel function of the 2nd kind.

In order write a function for the entire power spectrum of an electron we will need some more modules:

In [ ]:
import scipy.integrate as integrate
import scipy.special as special

The function $F(x)$ can be programmed as follows. (see https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html)

In [ ]:
def F(x) :
    ## https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html
    
    #K53 = special.kv(5./3, x)
    result = x*integrate.quad(lambda z: special.kv(5./3,z), x, np.inf)[0]
    
    return result

But we need something that will work on a vector of x values:

In [ ]:
def F_vec(x_vec) :
    
    integrateArray = []
    
    for x in x_vec :
        integrate = F(x)
        integrateArray.append(integrate)
    
    return integrateArray    

Here is a function to calculate the power radiated by an electron with energy $E$ in a magnetic field $B$ with a pitch angle $\phi$. nu can be a single frequency or an array of frequencies. Everything else must be a scalar.

In [ ]:
def Psync(nu, E, B, phi = 90*u.degree) : 

    nu_over_nus = nu / nu_sync (E, B, phi)
    
    nu_over_nus = nu_over_nus.decompose().value
    
    if type(nu_over_nus) is np.ndarray :
        print('array mode')
        ret = np.array(F_vec(nu_over_nus))*u.one
    else :
        print('single mode')
        ret = F(nu_over_nus)*u.one


    ret *= sqrt(3)/8/pi**2 
    
    ret *= c.e.si**3 * B * sin(phi) / c.m_e / c.c / c.eps0.si


    return ret.to(u.W/u.Hz)


Let's calulcate the power for a range of frequencies. Let's use an electron energy of $U = 19\,\mathrm{TeV}$ and $B = 5\times10^{-8}\,\mathrm{T}$. 

In [ ]:
U = 19*u.TeV
B = 5e-8*u.T

In [ ]:
frequencies = np.logspace(13, 19, 601) * u.Hz

In [ ]:
P = Psync(frequencies, U, B)

In [ ]:
P

Let's make a plot of it:

In [ ]:
plt.plot(frequencies, P)

plt.xscale('log')

plt.xlabel('frequency [{}]'.format(frequencies.unit))
plt.ylabel('P [{}]'.format(P.unit))

### Exercise
Make plots similar to the plots on slide 45 of the lecture. Use 0.1, 1 and 10 TeV for the electron energy and a magnetic field of $B = 10^{-8}\,\mathrm{T}$ for the first plot, and an electron energy of $U = 10\,\mathrm{TeV}$ and magnetic fields of $10^{-10}\,\mathrm{T}$,$10^{-9}\,\mathrm{T}$, and $10^{-8}\,\mathrm{T}$.

In [ ]:
# for plot 1
U_1 = [0.1, 1, 10] *u.TeV
B_1 = 1e-8 * u.T

# fill in your code here

In [ ]:
# for plot 2
U_2 = 10 *u.TeV
B_2 = [1e-10, 1e-9, 1e-8] * u.T

# fill in your code here

## Emitted Power in the Relativistic Case 

We will use the equation from slide 52 of the lecture. Electron energy and magnetic field are as above:

In [ ]:
U

In [ ]:
B

As the electron is highly relativistic we can set $\beta$ to $1$.

In [ ]:
beta = 1

In [ ]:
P = 2./3 * c.eps0.si * c.sigma_T * beta**2 / (c.c * c.m_e**2) * B**2 * U**2

In [ ]:
P.to(u.W)